# import libraries

In [1]:
# import important libraries
import pandas as pd
import numpy as np
import re
import nltk
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load dataset
df = pd.read_csv("Reviews.csv")
df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [3]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df.tail()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...
568453,568454,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0,0,5,1338422400,Great Honey,"I am very satisfied ,product is as advertised,..."


In [5]:
# understand each and every column in dataset
df.columns


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [6]:
df.shape

(568454, 10)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568428 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [8]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [9]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


# Data Preprocessing

In [11]:
#Drop unimportant columns
df.drop(columns=['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Time', 'Summary'], inplace=True)


In [12]:
df

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...
...,...,...
568449,5,Great for sesame chicken..this is a good if no...
568450,2,I'm disappointed with the flavor. The chocolat...
568451,5,"These stars are small, so you can give 10-15 o..."
568452,5,These are the BEST treats for training and rew...


In [13]:
# Step 1: Convert Ratings to Sentiment Labels
def sentiment_label(score):
    if score >= 3:
        return "Positive"
    else:
        return "Negative"

df['Sentiment'] = df['Score'].apply(sentiment_label)


In [14]:
df['Sentiment'].value_counts()

Sentiment
Positive    486417
Negative     82037
Name: count, dtype: int64

# Handling imbalance data
- #### Balance the dataset using undersampling

In [16]:
from sklearn.utils import resample

# Separate majority (Positive) and minority (Negative) classes
positive_reviews = df[df["Sentiment"] == "Positive"]
negative_reviews = df[df["Sentiment"] == "Negative"]

# Undersample Positive reviews to match Negative count
positive_sampled = resample(positive_reviews, 
                            replace=False, 
                            n_samples=len(negative_reviews), 
                            random_state=42)

# Combine balanced dataset
df = pd.concat([positive_sampled, negative_reviews]).sample(frac=1, random_state=42).reset_index(drop=True)

# Check new class balance
print(df["Sentiment"].value_counts())



Sentiment
Positive    82037
Negative    82037
Name: count, dtype: int64


In [18]:
df

,Score,Text,Sentiment
0,3,This morning blend is average flavor coffee. ...,Positive
1,5,I was turned onto this rice seven years ago wh...,Positive
2,5,This bag of Kirkland Mix hard candys was a big...,Positive
3,5,Have loved this since I was a little kid. Grea...,Positive
4,2,Kids did not like it --- lactose intolerant my...,Negative
...,...,...,...
164069,1,The cherries tasted rancid - DO NO BUY!!!!! W...,Negative
164070,2,I really wanted to like these nut bars. My mo...,Negative
164071,1,The Pennsylvania Dutch Candies Peanut Brittle ...,Negative
164072,1,I usually do not bother to write any reviews a...,Negative


# Text Cleaning

In [20]:
# Download necessary NLTK resources
nltk.download("punkt")
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z]", " ", text)  # Remove special characters
    words = word_tokenize(text)  # Tokenize words
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Apply Lemmatization
    return " ".join(processed_words)

# Apply preprocessing to all rows
df["Cleaned_Text"] = df["Text"].astype(str).apply(preprocess_text)

In [38]:
df

,Score,Text,Sentiment,Cleaned_Text
0,3,This morning blend is average flavor coffee. ...,Positive,morning blend average flavor coffee nothing sp...
1,5,I was turned onto this rice seven years ago wh...,Positive,turned onto rice seven year ago met husband fi...
2,5,This bag of Kirkland Mix hard candys was a big...,Positive,bag kirkland mix hard candy big hit office gre...
3,5,Have loved this since I was a little kid. Grea...,Positive,loved since little kid great flavor much bette...
4,2,Kids did not like it --- lactose intolerant my...,Negative,kid like lactose intolerant may fine boy cup t...
...,...,...,...,...
164069,1,The cherries tasted rancid - DO NO BUY!!!!! W...,Negative,cherry tasted rancid buy purchase dried cherry...
164070,2,I really wanted to like these nut bars. My mo...,Negative,really wanted like nut bar mouth started water...
164071,1,The Pennsylvania Dutch Candies Peanut Brittle ...,Negative,pennsylvania dutch candy peanut brittle taste ...
164072,1,I usually do not bother to write any reviews a...,Negative,usually bother write review think enough revie...


# Convert Text into Features (TF-IDF Vectorization)

In [40]:
# Convert Text to TF-IDF Features
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Cleaned_Text'])
y = df['Sentiment'].map({'Positive': 1, 'Negative': 0})


In [41]:
print(X)
print(y)

  (0, 5630)	0.28065470113554414
  (0, 3630)	0.20314648906293317
  (0, 22194)	0.18810746494674493
  (0, 13128)	0.24752092489208435
  (0, 5980)	0.18065558065241957
  (0, 57922)	0.24484677165007418
  (0, 49290)	0.28707668365885536
  (0, 35582)	0.2296231975705984
  (0, 10009)	0.3350139642622006
  (0, 19741)	0.13748707168911867
  (0, 3423)	0.3307906140909824
  (0, 5341)	0.5030236659235003
  (0, 33878)	0.24898777184879697
  (1, 54431)	0.05857048340504688
  (1, 7354)	0.08451385745107848
  (1, 36832)	0.09996620405521739
  (1, 30830)	0.048969269079018385
  (1, 1573)	0.05671301673446485
  (1, 14839)	0.1007085002110047
  (1, 56167)	0.07412489354308534
  (1, 22756)	0.04938594753647767
  (1, 47642)	0.08832749240973374
  (1, 50228)	0.1133165834179449
  (1, 5135)	0.06887218312768756
  (1, 8782)	0.16135528834006335
  :	:
  (164073, 9203)	0.23010709427764592
  (164073, 11325)	0.18550253427693056
  (164073, 53645)	0.15531195758918576
  (164073, 18513)	0.4099819153185867
  (164073, 44219)	0.2649882900882

# Train Naïve Bayes Model

In [43]:
# Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
# Train Naïve Bayes Model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [45]:
#Prediction
ypred_train=model.predict(X_train)
ypred_test=model.predict(X_test)

In [46]:
from sklearn.metrics import accuracy_score
print('train_accuracy:',accuracy_score(y_train,ypred_train))
print('test_accuracy:',accuracy_score(y_test,ypred_test))
print(classification_report(y_test, ypred_test))                         

train_accuracy: 0.8724582695281847
test_accuracy: 0.8519274721925949
              precision    recall  f1-score   support

           0       0.85      0.85      0.85     16382
           1       0.85      0.85      0.85     16433

    accuracy                           0.85     32815
   macro avg       0.85      0.85      0.85     32815
weighted avg       0.85      0.85      0.85     32815



# Save model

In [39]:
# Save Model and Vectorizer
with open("naive_bayes_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved successfully!")


Model and vectorizer saved successfully!
